<h1>Importing Libraries</h1>
<p>Importing necessary libraries like<br><blockquote>numpy<br>matplotlib.pyplot<br>pandas<br>pandas_datareader.data<br>datetime<br>yfinance<br>sklearn.preprocessing<br>keras</blockquote></p>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pandas_datareader.data as pdr
import datetime as dt
import yfinance as yf

from sklearn.preprocessing import MinMaxScaler
from keras.layers import Dense, Dropout, LSTM
from keras.models import Sequential

<h1>Data Collection</h1>
<p>Using the <i>yfinance</i> library to override <i>pandas</i> datareader’s method to fetch data</p>
<blockquote><code>yf.pdr_override()</code></blockquote>
<p>Defining the company ticker for fetching the data</p>
<blockquote><code>company = <i>ticker</i></code></blockquote>
<p>Setting the start and end dates for the data collection.</p>
<blockquote><code>start = dt.datetime(<i>starting date</i>)<br>end = dt.datetime(<i>end date</i>)<br><i>NOTE: use dt.datetime.now() for current date</i></code></blockquote>
<p>Using pandas_datareader to fetch the data from Yahoo Finance.</p>
<blockquote><code>data = pdr.get_data_yahoo(<i>ticker</i>, <i>starting date</i>, <i>ending date</i>)</code></blockquote>

In [ ]:
yf.pdr_override()

company = 'BTC-USD'

start = dt.datetime(2012,1,1)
end = dt.datetime.now()

data = pdr.get_data_yahoo(company, start=start, end=end)

<h1>Data Preprocessing</h1>
<p>Creating a MinMaxScaler to scale the closing prices of the company's stock unit. This scales the largest datapoint to the Max and the smallest datapoint to the Min</p>
<blockquote><code>scaler = MinMaxScaler(feature_range=(<i>minimum value</i>,<i> maximum value</i>))</code></blockquote>
<p>Transforming the ‘Close’ column values of the data to fit the scaler.</p>
<blockquote><code>scaled_data = scaler.fit_transform(data[<i>column name</i>].values.reshape(-1,1))</code></blockquote>

In [ ]:
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(data['Close'].values.reshape(-1,1))

<h1>Feature Engineering</h1>
<p>Defining a variable prediction_days as 60.</p>
<blockquote><code>prediction_days = <i>number of days</i></code></blockquote>

<p>Creating two lists, x_train and y_train, and appending the past 60 days’ scaled data to x_train and the next day’s data to y_train.</p>

<blockquote><code>x_train = []

y_train = []

for x in range(prediction_days, len(scaled_data)):
> x_train.append(scaled_data[x-prediction_days:x, 0])
<br>y_train.append(scaled_data[x, 0])</code></blockquote>

<p>Converting the lists to numpy arrays and reshaping x_train to fit the model.</p>
<blockquote><code>x_train, y_train = np.array(x_train), np.array(y_train)<br>
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))</code></blockquote>

In [ ]:
prediction_days = 60

x_train = []
y_train = []

for x in range(prediction_days, len(scaled_data)):
    x_train.append(scaled_data[x-prediction_days:x, 0])
    y_train.append(scaled_data[x, 0])


x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

<h1>Model Building</h1>
<p>Creating a Sequential model using Keras</p>
<blockquote><code>model = Sequential()</code></blockquote>
<p>Addition of three LSTM layers with 50 units each and a dropout of 0.2 after each LSTM layer to prevent overfitting.</p>
<blockquote><code>model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1], 1)))<br>
model.add(Dropout(0.2))<br>
model.add(LSTM(units=50, return_sequences=True))<br>
model.add(Dropout(0.2))<br>
model.add(LSTM(units=50))<br>
model.add(Dropout(0.2))</code></blockquote>
<p>Addition of a Dense layer with 1 unit at the end.</p>
<blockquote><code>model.add(Dense(units=1))</code></blockquote>

In [ ]:
model = Sequential()

model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units=1))

<h1>Model Compilation and Training</h1>
<p>Compiling the model with <i>‘adam’</i> optimizer and <i>‘mean_squared_error’</i> as the loss function.</p>
<blockquote><code>model.compile(optimizer = <i>optimizer type</i>, loss = <i>regression loss</i>)</code></blockquote>
<p>Fitting the model on x_train and y_train for 25 epochs with a batch size of 32.</p>
<blockquote><code>model.fit(x_train, y_train, epochs = <i>number of times to train</i>, batch_size = <i>size of batch</i>)</code></blockquote>

In [ ]:
model.compile(optimizer = 'adam', loss = 'mean_squared_error')
model.fit(x_train, y_train, epochs = 25, batch_size = 32)

<h1>Testing and Prediction</h1>
<p>Defining the start and end dates for the test data.</p>
<blockquote><code>test_start = dt.datetime(<i>starting date</i>)<br>
test_end = dt.datetime(<i>end date</i>)<br><i>NOTE: use dt.datetime.now() for current date</i></code></blockquote>
<p>Fetching the test data and storing the actual prices.</p>
<blockquote><code>test_data = pdr.get_data_yahoo(<i>ticker</i>, <i>starting date</i>, <i>ending date</i>)<br>
actual_prices = test_data[<i>column name</i>].values</code></blockquote>
<p>Preparing the model inputs by concatenating the <i>‘Close’</i> column of the data and test data.</p>
<blockquote><code>total_dataset = pd.concat((data[<i>column name</i>], test_data[<i>column name</i>]), axis=0)</code></blockquote>
<p>Transformation of the model inputs to fit the scaler.</p>
<blockquote><code>model_inputs = total_dataset[len(total_dataset) - len(test_data) - prediction_days:].values<br>
model_inputs = model_inputs.reshape(-1, 1)<br>
model_inputs = scaler.fit_transform(model_inputs)</code></blockquote>
<p>Creating a list <i>x_test</i> and appending the past 60 days’ model inputs to it.</p>
<blockquote><code>x_test = []<br>

for x in range(prediction_days, len(model_inputs)):
> x_test.append(model_inputs[x-prediction_days:x, 0])

x_test = np.array(x_test)<br>
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))</code></blockquote>
<p>Predicting the prices using the model and inverse transforming it to get the actual predicted prices.</p>
<blockquote><code>predicted_prices = model.predict(x_test)<br>
predicted_prices = scaler.inverse_transform(predicted_prices)</code></blockquote>

In [ ]:
test_start = dt.datetime(2023,7,25)
test_end = dt.datetime.now()


test_data = pdr.get_data_yahoo(company, start=start, end=end)
actual_prices = test_data['Close'].values

total_dataset = pd.concat((data['Close'], test_data['Close']), axis=0)


model_inputs = total_dataset[len(total_dataset) - len(test_data) - prediction_days:].values
model_inputs = model_inputs.reshape(-1, 1)
model_inputs = scaler.fit_transform(model_inputs)

x_test = []

for x in range(prediction_days, len(model_inputs)):
    x_test.append(model_inputs[x-prediction_days:x, 0])

x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))


predicted_prices = model.predict(x_test)
predicted_prices = scaler.inverse_transform(predicted_prices)

<h1>Visualization</h1>
<p>Plotting the actual and predicted prices using matplotlib.pyplot</p>

In [ ]:
plt.plot(actual_prices, color='black', label='Actual Prices')
plt.plot(predicted_prices, color='yellow', label='Predicted Prices')
plt.title(f'{company} price prediction')
plt.xlabel('Time')
plt.ylabel(f'{company} Price')
plt.legend(loc='upper left')
plt.show()

<h1>Future Prediction</h1>
<p>Preparation of the real data for the future prediction.</p>
<blockquote><code>real_data = [model_inputs[len(model_inputs) + 1 - prediction_days:len(model_inputs) + 1, 0]]<br>
real_data = np.array(real_data)<br>
real_data = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))</code></blockquote>
<p>Prediction of the next day’s closing price using the model and inverse transforming it to get the actual predicted price.</p>
<blockquote><code>prediction = model.predict(real_data)<br>
prediction = scaler.inverse_transform(prediction)<br>
print(f'prediction: {prediction}')</code></blockquote>

In [ ]:
real_data = [model_inputs[len(model_inputs) + 1 - prediction_days:len(model_inputs) + 1, 0]]
real_data = np.array(real_data)
real_data = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

prediction = model.predict(real_data)
prediction = scaler.inverse_transform(prediction)
print(f'prediction: {prediction}')